Дипломная работа "Классификация и оценка вирусности постов в социальных сетях"

Выполняется:
Студентом университета ИТМО кафедры Комьютерных Технологий Никитой Лисоветиным (https://github.com/turing228)

Под руководством:
Вячеслава Шаламова

### Полезные ссылки:
* https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

# Import Libraries

In [7]:
import vk_api
import time
import datetime
import json
import pandas as pd
import os

from PIL import Image
import requests
from io import BytesIO

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

# Connect To VK API

In [23]:
with open('Credentials.txt') as f:
    lines = f.readlines()

login = lines[0]
password = lines[1]

In [24]:
def auth_handler():
    """ При двухфакторной аутентификации вызывается эта функция.
    """

    # Код двухфакторной аутентификации
    key = input("Enter authentication code: ")
    # Если: True - сохранить, False - не сохранять.
    remember_device = True

    return key, remember_device

In [25]:
vk_session = vk_api.VkApi(login=login, password=password, auth_handler=auth_handler)
vk_session.auth()

vk = vk_session.get_api()

Enter authentication code: 432969


# Get Posts

In [30]:
domains = ['mudakoff']

In [31]:
domain = domains[0]

In [32]:
posts = vk.wall.get(domain=domain, count=100)

In [35]:
posts

{'count': 80159,
 'items': [{'id': 40301138,
   'from_id': -57846937,
   'owner_id': -57846937,
   'date': 1633250460,
   'marked_as_ads': 0,
   'post_type': 'post',
   'text': '',
   'is_pinned': 1,
   'attachments': [{'type': 'photo',
     'photo': {'album_id': -7,
      'date': 1633245241,
      'id': 460159961,
      'owner_id': -57846937,
      'has_tags': False,
      'access_key': '94c62024baa665e48e',
      'post_id': 40300840,
      'sizes': [{'height': 130,
        'url': 'https://sun1-84.userapi.com/impg/P-nVwhDVf0WjZFbt_evRj9b-fUz2QiZvb-DTVw/3hYZta4i89o.jpg?size=92x130&quality=96&sign=f41b6b28cabeddb56a71ed2ae4c1cfd5&c_uniq_tag=fWPoOMRaGp9Bep_E4PIbTVIzbcDGGxGCrpMzNuEM0dg&type=album',
        'type': 'm',
        'width': 92},
       {'height': 184,
        'url': 'https://sun1-84.userapi.com/impg/P-nVwhDVf0WjZFbt_evRj9b-fUz2QiZvb-DTVw/3hYZta4i89o.jpg?size=130x184&quality=96&sign=4af5c9a99689d6b947b6c528383f4e69&c_uniq_tag=583lyRyYbXv2NtnAmO2922Ulf7qRw6Q2UQKUscSNI5s&type=alb

In [67]:
posts['items'][0]

{'id': 40301138,
 'from_id': -57846937,
 'owner_id': -57846937,
 'date': 1633250460,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '',
 'is_pinned': 1,
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1633245241,
    'id': 460159961,
    'owner_id': -57846937,
    'has_tags': False,
    'access_key': '94c62024baa665e48e',
    'post_id': 40300840,
    'sizes': [{'height': 130,
      'url': 'https://sun1-84.userapi.com/impg/P-nVwhDVf0WjZFbt_evRj9b-fUz2QiZvb-DTVw/3hYZta4i89o.jpg?size=92x130&quality=96&sign=f41b6b28cabeddb56a71ed2ae4c1cfd5&c_uniq_tag=fWPoOMRaGp9Bep_E4PIbTVIzbcDGGxGCrpMzNuEM0dg&type=album',
      'type': 'm',
      'width': 92},
     {'height': 184,
      'url': 'https://sun1-84.userapi.com/impg/P-nVwhDVf0WjZFbt_evRj9b-fUz2QiZvb-DTVw/3hYZta4i89o.jpg?size=130x184&quality=96&sign=4af5c9a99689d6b947b6c528383f4e69&c_uniq_tag=583lyRyYbXv2NtnAmO2922Ulf7qRw6Q2UQKUscSNI5s&type=album',
      'type': 'o',
      'width': 130},
     {'height': 283,
    

In [115]:
datetime.datetime.utcfromtimestamp(1633250460).replace(tzinfo=datetime.timezone.utc).astimezone(tz=None)

datetime.datetime(2021, 10, 3, 11, 41, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800), 'Russia TZ 2 Standard Time'))

In [116]:
datetime.datetime.utcfromtimestamp(1633250460).replace(tzinfo=datetime.timezone.utc).astimezone(tz=None).strftime('%Y-%m-%d %H:%M:%S')

'2021-10-03 11:41:00'

In [117]:
def getPhotoUrl(post):
    attachments = post['attachments']
    if len(attachments) > 1:
        return ''
    
    if attachments[0]['type'] != 'photo':
        return ''
    
    big = {'height': 0, 'url': ''}
    for sizes in attachments[0]['photo']['sizes']:
        if sizes['height'] > big['height']:
            big['height'] = sizes['height']
            big['url'] = sizes['url']
            
    return big['url']

In [118]:
getPhotoUrl(posts['items'][2])

'https://sun1-56.userapi.com/impg/NYCMhemBLtPRfq3e3axfoX1-u1I90VqqkYQocA/oH0jWd_Ac1c.jpg?size=640x602&quality=96&sign=1530c005431de8549428164ca01d25de&c_uniq_tag=GJ501ywvJ4cLR8Opahed6pSHP3xUBJw6QGo0cS7w_84&type=album'

In [178]:
def getPostMetaInfo(post):
    return {
            'photo': getPhotoUrl(post),
#             'unix_timestamp': post['date'],
#             'datetime': datetime.datetime.utcfromtimestamp(post['date']),
            'moscow_date': datetime.datetime.utcfromtimestamp(post['date']).replace(tzinfo=datetime.timezone.utc).astimezone(tz=None).strftime('%Y-%m-%d %H:%M:%S'),
            'likes': post['likes']['count'],
            'reposts': post['reposts']['count'],
            'views': post['views']['count'],
            'comments': post['comments']['count']
           }

In [179]:
getPostMetaInfo(posts['items'][0])

{'photo': 'https://sun1-84.userapi.com/impg/P-nVwhDVf0WjZFbt_evRj9b-fUz2QiZvb-DTVw/3hYZta4i89o.jpg?size=1131x1600&quality=96&sign=8a6a19413ccd1f36dd98464c2b45f865&c_uniq_tag=Lz6utjKzumg17NWX-ZBijZYOER9uexFT0fUDz9JgOdg&type=album',
 'moscow_date': '2021-10-03 11:41:00',
 'likes': 8467,
 'reposts': 3430,
 'views': 432300,
 'comments': 230}

In [180]:
def getPostsMetaInfo(posts):
    data = []
    for item in posts['items']:
        metainfo = getPostMetaInfo(item)
        if metainfo['photo'] != '':
            data.append(getPostMetaInfo(item))
    
    return data

In [181]:
data = getPostsMetaInfo(posts)

In [182]:
with open('Posts.json', 'w') as f:
    json.dump({domain: data}, f)

In [183]:
data = {}
with open('Posts.json') as f:
    data = json.load(f)

In [184]:
data = data[domain]

In [185]:
df = pd.DataFrame(data)

In [186]:
df

,photo,moscow_date,likes,reposts,views,comments
0,https://sun1-84.userapi.com/impg/P-nVwhDVf0WjZ...,2021-10-03 11:41:00,8467,3430,432300,230
1,https://sun1-96.userapi.com/impg/G0D7aewqAwVft...,2021-10-03 21:39:00,340,136,35411,20
2,https://sun1-56.userapi.com/impg/NYCMhemBLtPRf...,2021-10-03 21:08:00,1109,289,74921,101
3,https://sun1-89.userapi.com/impg/cEOAppqj6qbgK...,2021-10-03 20:07:00,1452,806,127627,21
4,https://sun1-27.userapi.com/impg/xQ-kxo1FouGm3...,2021-10-03 19:40:00,2295,923,159350,37
...,...,...,...,...,...,...
68,https://sun1-24.userapi.com/impg/3cgvXwfIGUXbP...,2021-10-01 12:56:36,2512,1605,245582,93
69,https://sun1-85.userapi.com/impg/w_RKglzOxNX2B...,2021-10-01 12:10:00,7078,1583,523211,93
70,https://sun1-83.userapi.com/impg/HU7Pred0BLlwE...,2021-10-01 11:42:00,2803,1033,320106,53
71,https://sun1-96.userapi.com/impg/U994Md13PCtcK...,2021-10-01 11:10:00,3335,1637,395248,147


In [207]:
def loadImages(df):
    images = {}
    for i, row in df.iterrows():
        response = requests.get(row['photo'])
        img = Image.open(BytesIO(response.content))
        images[i] = img
    return images

In [208]:
images = loadImages(df[:3])

In [8]:
os.mkdir('memes')

In [210]:
for i in images:
    images[i].save('/memes/' + str(i), 'JPEG')

FileNotFoundError: [Errno 2] No such file or directory: '/memes/0'

In [205]:
images

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1131x1600 at 0x1FA122FF2B0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=654x605 at 0x1FA01EA7A60>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x602 at 0x1FA12365BB0>]

In [199]:
df.describe()

,likes,reposts,views,comments
count,73.000000,73.000000,73.000000,73.000000
mean,2804.863014,1371.068493,252802.698630,74.945205
std,1471.203639,837.466108,77448.927713,56.408700
min,340.000000,136.000000,35411.000000,13.000000
25%,1839.000000,869.000000,211345.000000,39.000000
50%,2492.000000,1220.000000,251274.000000,58.000000
75%,3410.000000,1753.000000,283609.000000,91.000000
max,8467.000000,4215.000000,523211.000000,295.000000


In [ ]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')